### Single Agent with State

In [ ]:
import os
import json
from pydantic import BaseModel, ValidationError
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent


# 1. Define schema
class ParsedIdeaModel(BaseModel):
    problem: str
    solution: str
    domain: str

class AgentState(BaseModel):
    raw_idea: str
    parsed_idea: ParsedIdeaModel = None

# 2. Model client setup

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="google/gemini-2.0-flash-001",
    api_key=open_router_api_key,
    model_info={
        "family": 'deepseek',
        "vision": True,
        "function_calling": True,
        "json_output": False,
        'structured_output': True
    }
)

# 3. Define the agent
agent = AssistantAgent(
    name='IdeaParserAgent',
    model_client=model_client,
    description="You are a startup idea validation assistant",
    system_message=(
        "You are an expert at understanding startup ideas. "
        "Given a raw idea, extract: (1) Problem, (2) Solution, (3) Domain. "
        "Return ONLY a JSON like: { 'problem': ..., 'solution': ..., 'domain': ... }"
    )
)

# 4. Task
task = "An AI-driven platform for personalized mental health support via chatbots."
result = await agent.run(task=task)

# 5. Extract and parse result
llm_response = result.messages[-1].content.strip()

# Remove markdown formatting if present
if llm_response.startswith("```json"):
    llm_response = llm_response.replace("```json", "").replace("```", "").strip()

try:
    parsed_dict = json.loads(llm_response)
    parsed = ParsedIdeaModel.model_validate(parsed_dict)

    # ✅ Create state instance and assign
    state = AgentState(raw_idea=task, parsed_idea=parsed)

    # ✅ Dump result
    print(state.model_dump_json(indent=2))

except (json.JSONDecodeError, ValidationError) as e:
    print("❌ Validation failed:", e)


### Multi Agent with State

In [ ]:
class ParsedIdeaModel(BaseModel):
    problem: str
    solution: str
    domain: str

class IdeaValidationModel(BaseModel):
    valid: bool
    explanation: str

class AgentState(BaseModel):
    raw_idea: str
    parsed_idea_model: ParsedIdeaModel = None
    idea_validation_model: IdeaValidationModel = None

agent_parse_idea = AssistantAgent(name="parse_idea",
                                  model_client=model_client,
                                  system_message=(
                                                    "You are an expert at understanding startup ideas. "
                                                    "Given a raw idea, extract: (1) Problem, (2) Solution, (3) Domain. "
                                                    "Return ONLY a JSON like: { 'problem': ..., 'solution': ..., 'domain': ... }"
                                                )
                                )

task = "An AI-driven platform for personalized mental health support via chatbots."

result = await agent_parse_idea.run(task=task)

idea_content = result.messages[-1].content
if idea_content:
    if 'json' in idea_content:
        idea_content = idea_content.replace("```json", "").replace("```", "").strip()
    
    parsed_data_json = json.loads(idea_content)
    state = AgentState(raw_idea=task)
    state = state.model_copy(update={
        "parsed_idea_model": parsed_data_json
    })

    print(state.model_dump_json(indent=2))

{'problem': 'Lack of access to affordable and personalized mental health support, leading to untreated mental health issues and suffering.', 'solution': 'An AI-powered chatbot platform that provides personalized mental health support, including screening, coping strategies, and connection to resources, available 24/7.', 'domain': 'Mental Health, Artificial Intelligence'}
{
  "raw_idea": "An AI-driven platform for personalized mental health support via chatbots.",
  "parsed_idea_model": {
    "problem": "Lack of access to affordable and personalized mental health support, leading to untreated mental health issues and suffering.",
    "solution": "An AI-powered chatbot platform that provides personalized mental health support, including screening, coping strategies, and connection to resources, available 24/7.",
    "domain": "Mental Health, Artificial Intelligence"
  },
  "idea_validation_model": null
}


C:\Users\Bhagwat\AppData\Roaming\Python\Python312\site-packages\pydantic\main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ParsedIdeaModel` - serialized value may not be as expected [input_value={'problem': 'Lack of acce...rtificial Intelligence'}, input_type=dict])
  return self.__pydantic_serializer__.to_json(


In [25]:
validation_prompt = """
You are a startup idea validation expert.

Given the following idea structure, validate whether the idea seems viable.
You must analyze: (1) Is the problem significant? (2) Is the solution feasible and valuable? (3) Is the domain promising?

Return a JSON like:
{
  "valid": true,
  "explanation": "Why the idea is valid or not"
}

Here is the idea:
Problem: {{problem}}
Solution: {solution}
Domain: {domain}
"""

# 1. Get parsed idea from previous step
parsed = state.parsed_idea_model

# 2. Format prompt
task_validation = validation_prompt.format(
    problem=state.parsed_idea_model['problem'],
    solution=state.parsed_idea_model['solution'],
    domain=state.parsed_idea_model['domain']
)

idea_validation_agent = AssistantAgent(name='idea_validation_agent',
                                       model_client=model_client,
                                       description="Evaluates whether the parsed startup idea is valid and promising",
                                       system_message="You are a startup evaluator. Return only JSON with keys: valid, explanation.")

result = await idea_validation_agent.run(task=task_validation)

validation_content = result.messages[-1].content

if validation_content:
    if '```json' in validation_content:
        validation_content = validation_content.replace("```json","").replace("```", "").strip()
    
    validation_content_json = json.loads(validation_content)

    state = state.model_copy(update={
        "idea_validation_model": validation_content_json
    })

    print(state.model_dump_json(indent=2))

KeyError: '\n  "valid"'

In [23]:
parsed['problem']

'Lack of access to affordable and personalized mental health support, leading to untreated mental health issues and suffering.'

In [20]:
state.parsed_idea_model['domain']

'Mental Health, Artificial Intelligence'